In [73]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)

In [4]:
RAW_FILE = '../../data/raw/W251 ML Covid Image Analysis_March 5, 2022_23.06.csv'
df = pd.read_csv(RAW_FILE, header=0, skiprows=[1,2])

Check the column names and find relevant fields only

In [6]:
df_cols = list(df.columns)
df_cols

['StartDate',
 'EndDate',
 'Status',
 'IPAddress',
 'Progress',
 'Duration (in seconds)',
 'Finished',
 'RecordedDate',
 'ResponseId',
 'RecipientLastName',
 'RecipientFirstName',
 'RecipientEmail',
 'ExternalReference',
 'LocationLatitude',
 'LocationLongitude',
 'DistributionChannel',
 'UserLanguage',
 'd_age_1',
 'd_gender',
 'd_vacc_status_1',
 'd_race',
 'd_race_6_TEXT',
 'v_temp_unit',
 'v_temp_c_1',
 'v_temp_f_1',
 'v_cough',
 's_antipyretic',
 's_odynophagia',
 's_odynophagia_days',
 's_dysphagia',
 's_dysphagia_days',
 't_rtpcr',
 't_rtpcr_date',
 't_ag',
 't_ag_date',
 't_med',
 'd_dx',
 'd_img1_Id',
 'd_img1_Name',
 'd_img1_Size',
 'd_img1_Type',
 'd_img2_Id',
 'd_img2_Name',
 'd_img2_Size',
 'd_img2_Type',
 'd_img3_Id',
 'd_img3_Name',
 'd_img3_Size',
 'd_img3_Type',
 'source']

We don't need most of the fields from Qualtrics except `ResponseId`. Also, pull the label column upfront for easy visualization

In [20]:

req_columns = ['d_dx','ResponseId']

d_cols = [c for c in df_cols if c[1] == '_' and c != 'd_dx'] 
req_columns.extend(d_cols)

In [25]:
df = df[req_columns]
print(f"DataFrame object now has {df.shape[0]} rows and {df.shape[1]} columns")
df.describe()

DataFrame object now has 142 rows and 33 columns


,d_age_1,d_vacc_status_1,v_temp_c_1,v_temp_f_1,s_odynophagia_days,s_dysphagia_days,d_img1_Size,d_img2_Size,d_img3_Size
count,140.000000,36.000000,135.000000,5.000000,11.000000,10.000000,1.380000e+02,3.000000e+01,1.800000e+01
mean,38.185714,2.166667,37.112593,98.460000,8.090909,8.800000,5.422573e+05,1.413368e+06,1.466108e+06
std,19.034080,0.941124,0.756211,0.134164,4.846742,5.006662,1.528889e+06,7.052928e+05,8.545069e+05
min,7.000000,0.000000,35.700000,98.400000,3.000000,2.000000,1.170200e+04,1.398000e+04,6.112500e+04
25%,23.000000,2.000000,36.600000,98.400000,5.000000,5.250000,2.043500e+04,9.810100e+05,9.584202e+05
50%,36.000000,2.000000,37.000000,98.400000,6.000000,8.500000,3.013150e+04,1.379062e+06,1.368224e+06
75%,50.250000,3.000000,37.500000,98.400000,12.000000,13.000000,8.084400e+05,1.650375e+06,1.615349e+06
max,91.000000,3.000000,40.200000,98.700000,16.000000,16.000000,1.618552e+07,3.548658e+06,3.571281e+06


In [107]:
def eda(df,col_names=None, row_width=60 ):
    columns = list(df.columns) if col_names is None else col_names
    print('='*row_width)
    print('df')
    print(f"{len(columns)} Reported \t{df.shape[1]} Columns\t{df.shape[0]} Rows" )
    for i,col in enumerate(columns):
        res = {}
        print('-'*row_width)
        dtype_name = df[col].dtype 
        res['n'] = df[col].count()
        res['NaN'] = df[col].isnull().sum()
        res['unique'] = len(df[col].unique())
        print(f"  {col}")
        d_col = pd.DataFrame([res])
        print(d_col.to_string(index=False))
        if res['unique'] < 15 or dtype_name == 'object':
            d_col_counts = df[col].value_counts(dropna=False)
            d_col_counts.name = 'Count'
            d_col_freq = df[col].value_counts(dropna=False, normalize=True)
            d_col_freq.name = 'Freq'
            df_col = pd.concat([d_col_counts,d_col_freq],axis=1)
            df_col = df_col.transpose()
            df_col.columns = [str(c)[:8] + ".." if len(str(c)) > 8 else str(c)[:8] for c in df_col.columns]
            if res['unique'] > 10:
                df_col_1 = df_col.iloc[:,:5]
                df_col_2 = df_col.iloc[:,-5:] 
                df_col = df_col_1.join(df_col_2, lsuffix='_1',rsuffix='_2')
            print(df_col.to_string())

    print('='*row_width)
            

In [110]:
eda(df)

df
33 Reported 	33 Columns	142 Rows
------------------------------------------------------------
  d_dx
   n  NaN  unique
 136    6       6
       Normal  Bacteria..   Covid  Viral    nan  Other
Count  53.000      49.000  24.000  7.000  6.000  3.000
Freq    0.373       0.345   0.169  0.049  0.042  0.021
------------------------------------------------------------
  ResponseId
   n  NaN  unique
 142    0     142
       R_Rylt07..  R_1E0ckj..  R_2b30KZ..  R_1dfXGn..  R_b3leKF..  R_1LkxHX..  R_1Cwqmo..  R_8J4qzo..  R_32Yi86..  R_2Se2TY..
Count       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000
Freq        0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007
------------------------------------------------------------
  d_age_1
   n  NaN  unique
 140    2      58
------------------------------------------------------------
  d_gender
   n  NaN  unique
 137    5 

In [38]:
eda(df)

object
object
float64
object
float64
object
object
object
float64
float64
object
object
object
float64
object
float64
object
object
object
object
object
object
object
float64
object
object
object
float64
object
object
object
float64
object
